LIMPIAR DATA TIKTOK

In [1]:
import pandas as pd
import re
import emoji
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

from nltk.corpus import wordnet, stopwords

In [2]:
#Carga de la data
tiktok_data = pd.read_csv('../data/tiktok_reviews.csv', low_memory=False)

In [3]:
tiktok_data.describe()

,score,thumbsUpCount
count,460287.000000,460287.000000
mean,4.234919,3.210601
std,1.430718,183.506791
min,1.000000,0.000000
25%,4.000000,0.000000
50%,5.000000,0.000000
75%,5.000000,0.000000
max,5.000000,40311.000000


In [4]:
#Tipos de datos
tiktok_data.dtypes

reviewId                object
userName                object
userImage               object
content                 object
score                    int64
thumbsUpCount            int64
reviewCreatedVersion    object
at                      object
replyContent            object
repliedAt               object
dtype: object

In [5]:
# Primeras filas 
tiktok_data.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,68ccaec8-1415-4301-a85e-3004679a3a83,Cassie Moore,https://play-lh.googleusercontent.com/a/ALm5wu...,No words,5,0,27.1.3,2022-11-29 21:55:37,NaN,NaN
1,d84cbfd3-6aa3-485c-aaf9-c5dca27dc966,Kaleb Plummer,https://play-lh.googleusercontent.com/a-/ACNPE...,Great fun app so far!,5,0,NaN,2022-11-29 21:55:04,NaN,NaN
2,96618aa1-31e5-4259-8649-89b75d962f00,Rylee Maher,https://play-lh.googleusercontent.com/a/ALm5wu...,The app would get a higher rating but I litera...,1,0,27.1.3,2022-11-29 21:54:48,NaN,NaN
3,078c0bda-598b-474e-a04e-d7cb3e6f6301,Kittykatelyn Romilly,https://play-lh.googleusercontent.com/a-/ACNPE...,I WISH I COULD GIVE THIS A 100 PERCENT RATING ...,5,0,NaN,2022-11-29 21:54:35,NaN,NaN
4,8e68c5cd-b12a-4206-a8da-6bfdbff44ae3,Loveness Malenga,https://play-lh.googleusercontent.com/a-/ACNPE...,Pictures and record,5,0,NaN,2022-11-29 21:54:21,NaN,NaN


In [6]:
# Ver información de la Data
tiktok_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460287 entries, 0 to 460286
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              460287 non-null  object
 1   userName              460287 non-null  object
 2   userImage             460287 non-null  object
 3   content               460256 non-null  object
 4   score                 460287 non-null  int64 
 5   thumbsUpCount         460287 non-null  int64 
 6   reviewCreatedVersion  333953 non-null  object
 7   at                    460287 non-null  object
 8   replyContent          260 non-null     object
 9   repliedAt             260 non-null     object
dtypes: int64(2), object(8)
memory usage: 35.1+ MB


In [7]:
# Verificar valores nulos
tiktok_data.isnull().sum()

reviewId                     0
userName                     0
userImage                    0
content                     31
score                        0
thumbsUpCount                0
reviewCreatedVersion    126334
at                           0
replyContent            460027
repliedAt               460027
dtype: int64

In [8]:
# Verificar si hay valores duplicados
tiktok_data.duplicated().sum()

np.int64(0)

In [9]:
#Eliminando algunas columnas sin mucha relevancia para el analisis
tiktok_data.drop(['userName'], axis=1, inplace=True)
tiktok_data.drop(['userImage'],axis=1, inplace=True)
tiktok_data.drop(['thumbsUpCount'], axis=1, inplace=True)
tiktok_data.drop(['reviewCreatedVersion'], axis=1, inplace=True)
tiktok_data.drop(['at'],axis=1, inplace=True)
tiktok_data.drop(['replyContent'], axis=1, inplace=True)
tiktok_data.drop(['repliedAt'], axis=1, inplace=True)

In [10]:
tiktok_data.columns

Index(['reviewId', 'content', 'score'], dtype='object')

In [11]:
# Verificar valores nulos
tiktok_data.isnull().sum()

reviewId     0
content     31
score        0
dtype: int64

In [14]:
# Eliminar valores nulos en content
tiktok_data = tiktok_data.dropna(subset=["content"])

In [15]:
tiktok_data.describe()

,score
count,460256.000000
mean,4.234919
std,1.430724
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [20]:
#antes de pasar a hacer la lematizacion hare una eliminacion de muchos datos positivos que hay(no hay de otra)
#eliminar muchos registros para balancear la data
condition = tiktok_data['score'] == 4
num_to_remove = 10000
rows_to_remove = tiktok_data[condition].head(num_to_remove)

# Eliminar esas filas del DataFrame original
tiktok_data = tiktok_data.drop(rows_to_remove.index)

tiktok_data.describe()

,score
count,112592.000000
mean,2.029372
std,1.333767
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,5.000000


In [21]:
conteo_sentimientos = tiktok_data.groupby('score').size()
conteo_sentimientos

score
1    61925
2    13179
3    18616
4    10000
5     8872
dtype: int64

In [22]:
# Convertir emojis a texto
def emojiText(texto):
    return emoji.demojize(texto, delimiters=(" ", " ")).replace("_", " ")

tiktok_data["content"] = tiktok_data["content"].apply(emojiText)

In [23]:
#funcion de lematizacion y tokenizacion
lemmatizer = WordNetLemmatizer()
# Obtener stopwords en inglés
stop_words = set(stopwords.words('english'))

customs_stop_wards = stop_words - {"not", "no", "never", "ever"}#dejamos habilitadas estas palabras

def tokenice_and_lemati(text):
    tokens = word_tokenize(text.lower())#convertimos todo a minusc y luego lo tokenizamos jijijij
    filter_tokens = [token for token in tokens if token not in customs_stop_wards]
    lematized_tokens = [lemmatizer.lemmatize(token, pos='v') for token in filter_tokens]
    return ' '.join(lematized_tokens)

In [24]:
#plematizar
tiktok_data['processed_text'] = tiktok_data['content'].apply(tokenice_and_lemati)

In [25]:
tiktok_data.head()

,reviewId,content,score,processed_text
2,96618aa1-31e5-4259-8649-89b75d962f00,The app would get a higher rating but I litera...,1,app would get higher rat literally ca n't sign...
8,0e4af7cc-53d9-4b81-b608-933af34dfae1,Kwai pelo menos da dinheiro,1,kwai pelo menos da dinheiro
9,25c6b61c-40ee-4361-9f01-dc25036bf076,Post to view is such a step backwards for the ...,1,post view step backwards platform -- punish ca...
10,f97768fe-323f-4422-9a60-e76781673ea0,I got banned for literally no reason. You guys...,1,get ban literally no reason . guy need stop ba...
21,66b4e691-a878-460b-8d62-239f756ec592,my app won't stop crashing its horrible I'm ma...,1,app wo n't stop crash horrible 'm make video u...


In [ ]:
#!pip install vaderSentiment

In [26]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def analizar_vader(texto):
    # Obtener el puntaje de polaridad
    sentiment_score = analyzer.polarity_scores(texto)
    compound_score = sentiment_score['compound']

    # Evaluar el sentimiento en base al puntaje "compound"
    if compound_score >= 0.5:
        return 'contento'  # Muy positivo
    elif compound_score >= 0.05:
        return 'positivo'  # Positivo
    elif compound_score <= -0.8:
        return 'enojado'  # Muy negativo
    elif compound_score <= -0.5:
        return 'insatisfecho'  #"enojado"
    elif compound_score <= -0.05:
        return 'negativo'  # Negativo
    else:
        return 'neutral'  # Neutral

In [27]:
tiktok_data['sentiment'] = tiktok_data['processed_text'].apply(analizar_vader)

In [35]:
conteo_sentimientos = tiktok_data.groupby('sentiment').size()
conteo_sentimientos

sentiment
enojado          3834
insatisfecho    11420
negativo        13535
neutral          4000
dtype: int64

In [34]:
#antes de pasar a hacer la lematizacion hare una eliminacion de muchos datos positivos que hay(no hay de otra)
#eliminar muchos registros para balancear la data
condition = tiktok_data['sentiment'] == 'neutral'
num_to_remove = 29362
rows_to_remove = tiktok_data[condition].head(num_to_remove)

# Eliminar esas filas del DataFrame original
tiktok_data = tiktok_data.drop(rows_to_remove.index)

tiktok_data.describe()

,score
count,32789.000000
mean,1.687975
std,1.189277
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,5.000000


In [36]:
# CSV limpio
tiktok_data.to_csv("../data/Tiktok_limpio.csv", index=False)